In [1]:
import os
import clang
from clang.cindex import *
from copy import deepcopy
import re

In [2]:
Config.set_library_file("/home/dipu/anaconda3/lib/python3.9/site-packages/clang/native/libclang.so")

In [3]:
"""
#include <stdio.h>
#include <stdlib.h>

int main()
{
    int aaa = 2, bbb = 3, ccc = 4, ddd = 5, eee, fff;
    int *zzz = &aaa;
    eee = aaa + bbb * ccc - ddd;
    fff = ccc * (aaa - bbb);
    
    printf("Result = %d", eee);
    return 0;
}
"""

'\n#include <stdio.h>\n#include <stdlib.h>\n\nint main()\n{\n    int aaa = 2, bbb = 3, ccc = 4, ddd = 5, eee, fff;\n    int *zzz = &aaa;\n    eee = aaa + bbb * ccc - ddd;\n    fff = ccc * (aaa - bbb);\n    \n    printf("Result = %d", eee);\n    return 0;\n}\n'

In [6]:
index = Index.create()
tu = index.parse("main.c")
root_cursor = tu.cursor

In [8]:
def print_ast(node, indent):
    try:
        current_name = "".join([node.spelling for x in list(node.get_tokens())]) if len(list(node.get_tokens())) > 0 else node.spelling
        # print(" "*indent + node.spelling + " " + str(node.kind) + " " + str(node.type.spelling))
        print(" "*indent + node.spelling + " " + str(node.kind))
        for c in node.get_children():
            print_ast(c, indent+2)
    except ValueError:
        pass

print_ast(root_cursor, 0)

main.c CursorKind.TRANSLATION_UNIT
  __u_char CursorKind.TYPEDEF_DECL
  __u_short CursorKind.TYPEDEF_DECL
  __u_int CursorKind.TYPEDEF_DECL
  __u_long CursorKind.TYPEDEF_DECL
  __int8_t CursorKind.TYPEDEF_DECL
  __uint8_t CursorKind.TYPEDEF_DECL
  __int16_t CursorKind.TYPEDEF_DECL
  __uint16_t CursorKind.TYPEDEF_DECL
  __int32_t CursorKind.TYPEDEF_DECL
  __uint32_t CursorKind.TYPEDEF_DECL
  __int64_t CursorKind.TYPEDEF_DECL
  __uint64_t CursorKind.TYPEDEF_DECL
  __int_least8_t CursorKind.TYPEDEF_DECL
    __int8_t CursorKind.TYPE_REF
  __uint_least8_t CursorKind.TYPEDEF_DECL
    __uint8_t CursorKind.TYPE_REF
  __int_least16_t CursorKind.TYPEDEF_DECL
    __int16_t CursorKind.TYPE_REF
  __uint_least16_t CursorKind.TYPEDEF_DECL
    __uint16_t CursorKind.TYPE_REF
  __int_least32_t CursorKind.TYPEDEF_DECL
    __int32_t CursorKind.TYPE_REF
  __uint_least32_t CursorKind.TYPEDEF_DECL
    __uint32_t CursorKind.TYPE_REF
  __int_least64_t CursorKind.TYPEDEF_DECL
    __int64_t CursorKind.TYPE_REF
 

                         CursorKind.UNEXPOSED_EXPR
                           CursorKind.UNEXPOSED_EXPR
                             CursorKind.PAREN_EXPR
                              __bsx CursorKind.DECL_REF_EXPR
                         CursorKind.INTEGER_LITERAL
                     CursorKind.INTEGER_LITERAL
               CursorKind.PAREN_EXPR
                 CursorKind.BINARY_OPERATOR
                   CursorKind.PAREN_EXPR
                     CursorKind.BINARY_OPERATOR
                       CursorKind.UNEXPOSED_EXPR
                         CursorKind.UNEXPOSED_EXPR
                           CursorKind.PAREN_EXPR
                            __bsx CursorKind.DECL_REF_EXPR
                       CursorKind.INTEGER_LITERAL
                   CursorKind.INTEGER_LITERAL
  __uint16_identity(__uint16_t) CursorKind.FUNCTION_DECL
    __uint16_t CursorKind.TYPE_REF
    __x CursorKind.PARM_DECL
      __uint16_t CursorKind.TYPE_REF
     CursorKind.COMPOUND_STMT
       CursorKind.RETU

In [6]:
# regex for matching string containing numbers, capital and small letters, +, -, *, /, %, _, parentheses and comma only
expression_pattern = re.compile(r"^[a-zA-Z0-9+\-*/%_(),]+$")

# regex for valid multiplication operator (not pointer)
mul_left_pattern = re.compile(r"^[a-zA-Z0-9_)]+$")
mul_right_pattern = re.compile(r"^[a-zA-Z0-9_(]+$")

In [7]:
def is_required_expression(node):
    # avoiding equals to ('=') operator
    if list(node.get_children())[0].kind == CursorKind.DECL_REF_EXPR:
        return False
    
    tokens = list(node.get_tokens())
    
    plus_minus_count = 0
    mul_div_mod_count = 0
    
    for i in range(len(tokens)):
        spell = str(tokens[i].spelling)
        if not expression_pattern.match(spell):
            return False
        
        if spell == "+" or spell == "-":
            plus_minus_count += 1
        elif spell == "/" or spell == "%":
            mul_div_mod_count += 1
        elif spell == "*" and 0 < i < len(tokens)-1 and \
        mul_left_pattern.match(str(tokens[i-1].spelling)) and \
        mul_right_pattern.match(str(tokens[i+1].spelling)):
            mul_div_mod_count += 1
            
    # expression with atleast two operators is needed
    if plus_minus_count + mul_div_mod_count < 2:
        return False
    
    return True

In [8]:
def get_binary_expressions(node, exp_list):
    try:
        if node.kind == CursorKind.BINARY_OPERATOR and is_required_expression(node):
            tokens = list(node.get_tokens())
            expression = [token.spelling for token in tokens]
            exp_list.append(expression)
        else:
            for child in node.get_children():
                get_binary_expressions(child, exp_list)
                
    except Exception as e:
        #print("***Exception***", e)
        pass

In [9]:
exp_list = []
get_binary_expressions(root_cursor, exp_list)
print(exp_list)

[['aaa', '+', 'bbb', '*', 'ccc', '-', 'ddd'], ['ccc', '*', '(', 'aaa', '-', 'bbb', ')']]


---
## Using C Code Corpus Dataset
---

In [10]:
def generate_binary_operator_expression_dataset(root_dir):
    total_files, total_samples = 0, 0
    global current_file
    
    with open("binary_operator_expression_dataset.csv", 'a') as dataset:
        
        for root, dirs, files in os.walk(root_dir):
                for file in files:
                    if file.endswith(".c"):
                        total_files += 1

                        file_path = os.path.join(root, file)
                        current_file = file_path
                            
                        with open(file_path, 'rb') as f:
                                
                            content = str(f.read())

                            # ignoring .c files having more than 10,000 lines of code
                            if content.count("\\n") <= 10_000:

                                try:
                                    start_cursor = index.parse(file_path).cursor

                                    expression_list = []
                                    get_binary_expressions(start_cursor, expression_list)

                                    for expression in expression_list:
                                        positive_sample = "\t".join(expression)
                                        dataset.write(positive_sample + "\n")

                                        total_samples += 1

                                except:
                                        print("---Error occurred---")

                        current_file = file_path

                        if total_files % 1000 == 0:
                            print("Total files:", total_files, ",", "Total samples:", total_samples)

In [11]:
root_dir = '/home/dipu/Documents/AI/MinorProject/c-corpus/'

generate_binary_operator_expression_dataset(root_dir)